In [1]:
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.linear_model import LinearRegression
import json
import csv
from sklearn import metrics
from tflite_benchmark_executor import TfLiteBenchmarkExecutor

In [2]:
with open('kernels.json') as json_file:
    kernels = json.load(json_file)

In [3]:
with open('squezzenet_layers.txt') as json_file:
    data = json.load(json_file)
    
    names = []
    for layer in data:
        names.append(layer)
        
    itens = []
    for name in names:
        itens.append(data[name])

In [4]:
    conv_2d = [[],[],[],[],[],[],[],[],[],[],[]]
    max_pol = [[],[],[],[],[],[],[],[],[],[],[]]
    
    conv_2d_time = [[],[],[],[],[],[],[],[],[],[],[]]
    
    for kernel,layer in zip(kernels,itens):
        
        if(kernel["kernel"] == "CONV_2D"):
        
            conv_2d[0].extend(np.repeat(layer["output"][0], len(kernel['energySpent'])))
            conv_2d[1].extend(np.repeat(layer["output"][1], len(kernel['energySpent'])))
            conv_2d[2].extend(np.repeat(layer["output"][2], len(kernel['energySpent'])))
            conv_2d[3].extend(np.repeat(layer["filters"][0], len(kernel['energySpent'])))
            conv_2d[4].extend(np.repeat(layer["filters"][1], len(kernel['energySpent'])))
            conv_2d[5].extend(np.repeat(layer["filters"][2], len(kernel['energySpent'])))
            conv_2d[6].extend(np.repeat(layer["stride"], len(kernel['energySpent'])))
            conv_2d[7].extend(np.repeat(layer["input"][0], len(kernel['energySpent'])))
            conv_2d[8].extend(np.repeat(layer["input"][1], len(kernel['energySpent'])))
            conv_2d[9].extend(np.repeat(layer["input"][2], len(kernel['energySpent'])))
            conv_2d[10].extend(np.asarray(kernel['energySpent']))
    
            conv_2d_time[0].extend(np.repeat(layer["output"][0], len(kernel['executionTime'])))
            conv_2d_time[1].extend(np.repeat(layer["output"][1], len(kernel['executionTime'])))
            conv_2d_time[2].extend(np.repeat(layer["output"][2], len(kernel['executionTime'])))
            conv_2d_time[3].extend(np.repeat(layer["filters"][0], len(kernel['executionTime'])))
            conv_2d_time[4].extend(np.repeat(layer["filters"][1], len(kernel['executionTime'])))
            conv_2d_time[5].extend(np.repeat(layer["filters"][2], len(kernel['executionTime'])))
            conv_2d_time[6].extend(np.repeat(layer["stride"], len(kernel['executionTime'])))
            conv_2d_time[7].extend(np.repeat(layer["input"][0], len(kernel['executionTime'])))
            conv_2d_time[8].extend(np.repeat(layer["input"][1], len(kernel['executionTime'])))
            conv_2d_time[9].extend(np.repeat(layer["input"][2], len(kernel['executionTime'])))
            conv_2d_time[10].extend(np.asarray(kernel['executionTime']))

            continue
    training_conv = []
    evaluating_conv = []
    
    conv_2d = np.transpose(conv_2d)
    
    np.random.shuffle(conv_2d)
    
    conv_2d = np.transpose(conv_2d)

    percentage = 0.8
    length = round(percentage * len(conv_2d[0]))
    
    for column in conv_2d:
        training_conv.append(column[:length])
        evaluating_conv.append(column[length:])

    training_y = np.asarray(training_conv[10])
    evaluating_y = np.asarray(evaluating_conv[10])
    
    training_conv = np.asarray(training_conv)
    evaluating_conv = np.asarray(evaluating_conv)
    
    training_conv = training_conv[:-1]
    evaluating_conv = evaluating_conv[:-1]
    
    print(len(training_y))

44928


In [5]:
    training_conv_time = []
    evaluating_conv_time = []
    
    conv_2d_time = np.transpose(conv_2d_time)
    
    np.random.shuffle(conv_2d_time)
    
    conv_2d_time = np.transpose(conv_2d_time)

    percentage = 0.8
    length = round(percentage * len(conv_2d_time[0]))
    
    for column in conv_2d_time:
        training_conv_time.append(column[:length])
        evaluating_conv_time.append(column[length:])
        
    training_y_time = np.asarray(training_conv_time[10])
    evaluating_y_time = np.asarray(evaluating_conv_time[10])
    
    training_conv_time = np.asarray(training_conv_time)
    evaluating_conv_time = np.asarray(evaluating_conv_time)
    
    training_conv_time = training_conv_time[:-1]
    evaluating_conv_time = evaluating_conv_time[:-1]

In [6]:
    training_conv = np.transpose(training_conv)
    evaluating_conv = np.transpose(evaluating_conv)
    
    reg = LinearRegression()
    reg.fit(training_conv, training_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [7]:
    training_conv_time = np.transpose(training_conv_time)
    evaluating_conv_time = np.transpose(evaluating_conv_time)
    
    reg_time = LinearRegression()
    reg_time.fit(training_conv_time, training_y_time)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [8]:
    y_pred = reg.predict(evaluating_conv)
    y_pred_time = reg_time.predict(evaluating_conv_time)

In [9]:
    print('Intercept: \n', reg.intercept_)
    print('Coefficients: \n', reg.coef_)
    
    print('Intercept: \n', reg_time.intercept_)
    print('Coefficients: \n', reg_time.coef_)

Intercept: 
 -4728312.39596551
Coefficients: 
 [-2.29329681e+07  2.29751852e+07  1.22415963e+09 -1.22415963e+09
 -2.54472640e+04  2.54473354e+04  4.72831225e+06 -2.96013877e+06
  2.91792169e+06  1.30830333e-04]
Intercept: 
 6132098483.106139
Coefficients: 
 [-9.14768144e+11  9.14713393e+11  4.53431235e+11 -4.53431235e+11
 -1.35991470e+08  1.35991493e+08 -6.13209853e+09  3.97541766e+09
 -3.92066678e+09  4.14428711e-02]


In [10]:
    reg.score(evaluating_conv, evaluating_y)

0.9121346558941082

In [11]:
    reg_time.score(evaluating_conv_time, evaluating_y_time)

0.9301579871576299

In [12]:
    print('Mean Absolute Error:', metrics.mean_absolute_error(evaluating_y, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(evaluating_y, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(evaluating_y, y_pred)))

Mean Absolute Error: 0.025323835280503165
Mean Squared Error: 0.0015756114536055485
Root Mean Squared Error: 0.03969397250975957


In [13]:
    print('Mean Absolute Error:', metrics.mean_absolute_error(evaluating_y_time, y_pred_time))
    print('Mean Squared Error:', metrics.mean_squared_error(evaluating_y_time, y_pred_time))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(evaluating_y_time, y_pred_time)))

Mean Absolute Error: 7.6806363062532546
Mean Squared Error: 143.06441374580976
Root Mean Squared Error: 11.960953713889614
